# Multivariate Linear Regression

## 이론적 개요

다음과 같은 훈련 데이터가 있습니다.     
앞서 배운 단순 선형 회귀와 다른 점은 독립 변수 의 개수가 이제 1개가 아닌 3개라는 점입니다.     
3개의 퀴즈 점수로부터 최종 점수를 예측하는 모델을 만들어보겠습니다.   

<img src=./LinearRegression_01.png>

독립 변수$x$ 의 개수가 3개므로 이를 수식으로 표현하면 아래와 같습니다

$ H(x) = x_1w_1 + x_2w_2 + x_3w_3 + b $   

$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $   

 - $H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가 ?
 - $cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가 ?

## Imports

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
# For reproducibility
torch.manual_seed(1)

## 1.3  Naive Data Representation

$ H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3 + b $ 

위의 식을 보면 이번에는 단순 선형 회귀와 다르게  𝑥  의 개수가 3개입니다. 그러니까 를 3개 선언합니다.

In [10]:
# 데이터
x1_train = torch.FloatTensor([[73],
                              [93],
                              [89],
                              [96],
                              [73]])
x2_train = torch.FloatTensor([[80],
                              [88],
                              [91],
                              [98],
                              [66]])
x3_train = torch.FloatTensor([[75],
                              [93],
                              [90],
                              [100],
                              [70]])
y_train = torch.FloatTensor([[152],
                             [185],
                             [180],
                             [196],
                             [142]])

이제 가중치 $W$ 와 편향 $b$ 를 선언합니다. 가중치 $W$ 도 3개 선언해주어야 합니다.

In [11]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [12]:
print("w1 :", w1.item())
print("w2 :", w2.item())
print("w2 :", w2.item())
print("b  :", b.item())

w1 : 0.0
w2 : 0.0
w2 : 0.0
b  : 0.0


In [13]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.294 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.661 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.655 w3: 0.677 b: 0.008 Cost: 1.497607
Epoch  300/1000 w1: 0.684 w2: 0.649 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.643 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.638 w3: 0.678 b: 0.009 Cost: 1.319511
Epoch  600/1000 w1: 0.699 w2: 0.633 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.627 w3: 0.679 b: 0.009 Cost: 1.215696
Epoch  800/1000 w1: 0.709 w2: 0.622 w3: 0.679 b: 0.009 Cost: 1.167818
Epoch  900/1000 w1: 0.713 w2: 0.617 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.613 w3: 0.680 b: 0.009 Cost: 1.079378


위의 경우 가설을 선언하는 부분인 hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b에서도     
x_train의 개수만큼 w와 곱해주도록 작성해준 것을 확인할 수 있습니다.

In [14]:
pred = (73*0.718 +  80* 0.613 +  75 * 0.680) 
print(pred)

152.454


$H(X) = w_{1}x_{1} + w_{2}x_{2} + w_{3}x_{3}$

위 식은 아래와 같이 두 벡터의 내적으로 표현할 수 있습니다.

$
\begin{pmatrix}
x_1 & x_2 & x_3
\end{pmatrix}
\cdot
\begin{pmatrix}
w_1 \\
w_2 \\
w_3 \\
\end{pmatrix}
=
\begin{pmatrix}
x_1w_1 + x_2w_2 + x_3w_3
\end{pmatrix}
$

두 벡터를 각각 $X$와 $W$로 표현한다면, 가설은 다음과 같습니다.

$ H(X) = XW $

$x$의 개수가 3개였음에도 이제는 $X$와 $W$라는 두 개의 변수로 표현된 것을 볼 수 있습니다.

In [15]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152],
                             [185],
                             [180],
                             [196],
                             [142]])

이는 독립 변수 $x$ 들의 수가 (샘플의 수 × 특성의 수) = 15개임을 의미합니다. 독립 변수 $x$들을 (샘플의 수 × 특성의 수)의 크기를 가지는 하나의 행렬로 표현해봅시다. 그리고 이 행렬을 $X$라고 하겠습니다.

$\begin{pmatrix}
    \begin{array}{c}
      x_{11}\ x_{12}\ x_{13}\ \\
      x_{21}\ x_{22}\ x_{23}\ \\
      x_{31}\ x_{32}\ x_{33}\ \\
      x_{41}\ x_{42}\ x_{43}\ \\
      x_{51}\ x_{52}\ x_{53}\ \\
    \end{array}
\end{pmatrix}$

그리고 여기에 가중치 $w1, w2, w3$ 을 원소로 하는 벡터를 $X$ 라 하고 이를 곱해보겠습니다.

$\begin{pmatrix}
   \begin{array}{c}
     x_{11}\ x_{12}\ x_{13}\ \\
     x_{21}\ x_{22}\ x_{23}\ \\
     x_{31}\ x_{32}\ x_{33}\ \\
     x_{41}\ x_{42}\ x_{43}\ \\
     x_{51}\ x_{52}\ x_{53}\ \\
   \end{array}
\end{pmatrix}
$
$\begin{pmatrix}
    \begin{array}{c}
      w_{1} \\
      w_{2} \\
      w_{3} \\
    \end{array}
\end{pmatrix}$
\ =
$\begin{pmatrix}
    \begin{array}{c}
      x_{11}w_{1}+ x_{12}w_{2}+ x_{13}w_{3}\ \\
      x_{21}w_{1}+ x_{22}w_{2}+ x_{23}w_{3}\ \\
      x_{31}w_{1}+ x_{32}w_{2}+ x_{33}w_{3}\ \\
      x_{41}w_{1}+ x_{42}w_{2}+ x_{43}w_{3}\ \\
      x_{51}w_{1}+ x_{52}w_{2}+ x_{53}w_{3}\ \\
    \end{array}
\end{pmatrix}$

위의 식은 결과적으로 다음과 같습니다.   
   
$H(X) = XW$

이 가설에 각 샘플에 더해지는 편향 $b$ 를 추가해봅시다. 샘플 수만큼의 차원을 가지는 편향 벡터$B$ 를 만들어 더합니다.

$\begin{pmatrix}
    \begin{array}{c}
      x_{11}\ x_{12}\ x_{13}\ \\
      x_{21}\ x_{22}\ x_{23}\ \\
      x_{31}\ x_{32}\ x_{33}\ \\
      x_{41}\ x_{42}\ x_{43}\ \\
      x_{51}\ x_{52}\ x_{53}\ \\
    \end{array}
\end{pmatrix}$
$\begin{pmatrix}
    \begin{array}{c}
      w_{1} \\
      w_{2} \\
      w_{3} \\
    \end{array}
\end{pmatrix}$
$+$
$\begin{pmatrix}
    \begin{array}{c}
      b \\
      b \\
      b \\
      b \\
      b \\
    \end{array}
\end{pmatrix}$
$\ =$
$\begin{pmatrix}
    \begin{array}{c}
      x_{11}w_{1}+ x_{12}w_{2}+ x_{13}w_{3} + b\ \\
      x_{21}w_{1}+ x_{22}w_{2}+ x_{23}w_{3} + b\ \\
      x_{31}w_{1}+ x_{32}w_{2}+ x_{33}w_{3} + b\ \\
      x_{41}w_{1}+ x_{42}w_{2}+ x_{43}w_{3} + b\ \\
      x_{51}w_{1}+ x_{52}w_{2}+ x_{53}w_{3} + b\ \\
    \end{array}
\end{pmatrix}$

위의 식은 결과적으로 다음과 같습니다.

$H(X) = XW + B$

결과적으로 전체 훈련 데이터의 가설 연산을 3개의 변수만으로 표현하였습니다.   
이와 같이 벡터와 행렬 연산은 식을 간단하게 해줄 뿐만 아니라 다수의 샘플의 병렬 연산이므로 속도의 이점을 가집니다.

In [16]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [17]:
# 가중치 w와 편향 b 초기화
W = torch.zeros((3,1), requires_grad=True)

b = torch.zeros(1, requires_grad=True)

In [21]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 30

for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {}  Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))

Epoch    0/30 hypothesis: tensor([152.7939, 183.6813, 180.9665, 197.0692, 140.1102])  Cost: 1.603586
Epoch    1/30 hypothesis: tensor([152.7935, 183.6816, 180.9664, 197.0691, 140.1106])  Cost: 1.602878
Epoch    2/30 hypothesis: tensor([152.7931, 183.6819, 180.9663, 197.0690, 140.1110])  Cost: 1.602179
Epoch    3/30 hypothesis: tensor([152.7926, 183.6822, 180.9661, 197.0689, 140.1114])  Cost: 1.601486
Epoch    4/30 hypothesis: tensor([152.7922, 183.6826, 180.9660, 197.0688, 140.1118])  Cost: 1.600765
Epoch    5/30 hypothesis: tensor([152.7918, 183.6828, 180.9659, 197.0686, 140.1122])  Cost: 1.600072
Epoch    6/30 hypothesis: tensor([152.7914, 183.6831, 180.9657, 197.0685, 140.1126])  Cost: 1.599369
Epoch    7/30 hypothesis: tensor([152.7909, 183.6834, 180.9656, 197.0684, 140.1130])  Cost: 1.598692
Epoch    8/30 hypothesis: tensor([152.7905, 183.6837, 180.9655, 197.0683, 140.1134])  Cost: 1.597991
Epoch    9/30 hypothesis: tensor([152.7900, 183.6840, 180.9653, 197.0682, 140.1138])  Cost:

In [25]:
x_train = torch.FloatTensor([[1],
                             [2],
                             [3],
                             [4],
                             [5]])
y_train = torch.FloatTensor([[2],
                             [4],
                             [6],
                             [8],
                             [10]])

In [26]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [32]:
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 100

for epoch in range(nb_epochs + 1):
    
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(hypothesis , y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {}  Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))

Epoch    0/100 hypothesis: tensor([-0.0245,  0.3408,  0.7060,  1.0712,  1.4364])  Cost: 33.371571
Epoch    1/100 hypothesis: tensor([0.4644, 1.2127, 1.9610, 2.7092, 3.4575])  Cost: 19.447556
Epoch    2/100 hypothesis: tensor([0.8376, 1.8783, 2.9190, 3.9596, 5.0003])  Cost: 11.333327
Epoch    3/100 hypothesis: tensor([1.1225, 2.3864, 3.6503, 4.9142, 6.1781])  Cost: 6.604755
Epoch    4/100 hypothesis: tensor([1.3399, 2.7742, 4.2086, 5.6429, 7.0772])  Cost: 3.849176
Epoch    5/100 hypothesis: tensor([1.5058, 3.0703, 4.6347, 6.1992, 7.7636])  Cost: 2.243360
Epoch    6/100 hypothesis: tensor([1.6325, 3.2963, 4.9601, 6.6238, 8.2876])  Cost: 1.307568
Epoch    7/100 hypothesis: tensor([1.7291, 3.4688, 5.2084, 6.9480, 8.6876])  Cost: 0.762231
Epoch    8/100 hypothesis: tensor([1.8029, 3.6004, 5.3980, 7.1955, 8.9930])  Cost: 0.444436
Epoch    9/100 hypothesis: tensor([1.8591, 3.7009, 5.5427, 7.3844, 9.2262])  Cost: 0.259238
Epoch   10/100 hypothesis: tensor([1.9021, 3.7776, 5.6531, 7.5286, 9.404

In [ ]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152],
                             [185],
                             [180],
                             [196],
                             [142]])

In [22]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [24]:
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20

for epoch in range(nb_epochs + 1):
    
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(hypothesis , y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {}  Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))

Epoch    0/20 hypothesis: tensor([-6.7933, -4.8968, -6.5155, -7.3361, -2.6660])  Cost: 31667.599609
Epoch    1/20 hypothesis: tensor([62.7036, 78.6330, 75.7880, 82.2903, 61.0462])  Cost: 9926.266602
Epoch    2/20 hypothesis: tensor([101.6124, 125.3983, 121.8667, 132.4688,  96.7163])  Cost: 3111.513916
Epoch    3/20 hypothesis: tensor([123.3960, 151.5805, 147.6645, 160.5620, 116.6867])  Cost: 975.451355
Epoch    4/20 hypothesis: tensor([135.5919, 166.2389, 162.1078, 176.2903, 127.8673])  Cost: 305.908539
Epoch    5/20 hypothesis: tensor([142.4200, 174.4456, 170.1940, 185.0960, 134.1269])  Cost: 96.042496
Epoch    6/20 hypothesis: tensor([146.2428, 179.0401, 174.7213, 190.0260, 137.6314])  Cost: 30.260748
Epoch    7/20 hypothesis: tensor([148.3830, 181.6125, 177.2559, 192.7861, 139.5934])  Cost: 9.641701
Epoch    8/20 hypothesis: tensor([149.5814, 183.0526, 178.6749, 194.3314, 140.6919])  Cost: 3.178671
Epoch    9/20 hypothesis: tensor([150.2523, 183.8588, 179.4694, 195.1966, 141.3068]) 